# Preprocessing

### Cropping Video

The video I downloaded has the footage on the right half, so I'm cropping the left half out.

In [149]:
import cv2
def crop_right_half(input_filename, input_folder="data/Raw Videos", output_folder="data/Final Videos"):
    """
    Crops the right half of the video from the specified input folder
    and saves the result to the specified output folder.
    
    Args:
        input_filename (str): The name of the input video file.
        input_folder (str): The folder containing raw videos.
        output_folder (str): The folder to save cropped videos.
    """
    import os
    import cv2

    input_path = os.path.join(input_folder, input_filename)
    output_path = os.path.join(output_folder, input_filename)

    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    full_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    half_width = full_width // 2

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (half_width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        right_half = frame[:, full_width//2:]
        out.write(right_half)

    cap.release()
    out.release()
    print(f"Cropped video saved to {output_path}")

In [150]:
crop_right_half("adl-30-cam0.mp4")

Cropped video saved to data/Final Videos/adl-30-cam0.mp4


In [166]:
import ultralytics
ultralytics.__file__

'/Users/sanjanamohan/Documents/Github/Fall_Detection/.venv/lib/python3.11/site-packages/ultralytics/__init__.py'

# Fall Detection

### YOLOv8-pose

Detecting fall based on drop in center of gravity

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import shutil
import math
import os
import torch
import json

In [3]:
mps_device = torch.device('mps')

In [4]:
from ultralytics.trackers import BYTETracker
# Configure tracker
tracker_params = {
    "max_age": 60,
    "min_hits": 3,
    "iou_threshold": 0.5,
    "match_thresh": 0.9,
}
model = YOLO("yolov8l-pose.pt").to(mps_device)
# model.to(mps_device)

In [ ]:
def run_fall_detection(input_video_path,min_area = 2000):
    """
    Runs fall detection on a given video and writes the annotated output to the 'Outputs' folder.

    Args:
        input_video_path (str): Path to the input video file.
        output_video_name (str): (Optional) Name of the output video file. Defaults to 'fall_detection_output.mp4'.

    Returns:
        str: Path to the saved annotated video.
    """

    # Ensure Outputs directory exists
    os.makedirs("Outputs", exist_ok=True)
    input_video_filename = os.path.basename(input_video_path)
    output_path = os.path.join("Outputs", input_video_filename)
    people_images_folder = os.path.splitext(input_video_filename)[0]
    people_output_dir = os.path.join('Outputs', people_images_folder + "_people")
    if os.path.exists(people_output_dir):
        shutil.rmtree(people_output_dir)
    os.makedirs(people_output_dir)

    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS) or 20
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        output_path,
        fourcc,
        fps,
        (width, height)
    )

    people = {}
    prev_timestamp = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_org = frame.copy()
        frame_height, frame_width = frame.shape[:2]
        # results = model(frame)
        results = model.track(frame, persist=True, tracker='./configs/bytetrack.yaml' , conf=0.35)

        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        timestamp_sec = timestamp_ms / 1000.0
        dt = timestamp_sec - prev_timestamp
        prev_timestamp = timestamp_sec
        print(dt)

        # Get current frame position (0-indexed)
        frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
        print(f"Frame ID: {frame_id}")
        fall_ongoing = False

        for result in results:
            if result.keypoints is None:
                continue

            keypoints = result.keypoints.xy.cpu().numpy()
            boxes = result.boxes.xyxy.cpu().numpy()
            ids = result.boxes.id
            if ids is None:
                continue
            ids = ids.cpu().numpy()
            # print(ids)

            for i, kps in enumerate(keypoints):
                box = boxes[i]
                person_id = int(ids[i])
                x1, y1, x2, y2 = map(int, box)
                area = (x2 - x1) * (y2 - y1)

                if area < min_area:
                    continue

                left_shoulder = kps[5]
                right_shoulder = kps[6]
                left_hip = kps[11]
                right_hip = kps[12]
                nose = kps[0]
                left_eye = kps[1]
                right_eye = kps[2]

                head_y = np.mean([nose[1], left_eye[1], right_eye[1]])

                # Compute torso center
                shoulder_center = (left_shoulder + right_shoulder) / 2
                hip_center = (left_hip + right_hip) / 2

                # Identifying and storing info about the people in the frame
                if person_id not in people:
                    # INSERT_YOUR_CODE
                    frame_filename = os.path.join(people_output_dir, f"{person_id}_{int(frame_id)}.jpg")
                    person_image = frame_org.copy()
                    # Draw bounding box & person
                    cv2.rectangle(person_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.circle(person_image, tuple(shoulder_center.astype(int)), 5, (255,0,0), -1)
                    cv2.circle(person_image, tuple(hip_center.astype(int)), 5, (0,255,0), -1)
                    cv2.putText(
                            person_image,
                            str(person_id),
                            (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.9, (255,255,255), 2
                        )
                    print(frame_filename)
                    saved = cv2.imwrite(frame_filename, person_image)
                    print(saved)
                    people[person_id] = {
                        'falls': [],
                        'frames':[],
                        'fall_ongoing': False,
                        'fall_frame':0,
                        'fall_done':0,
                        'angle_changed':False,
                        'prev_angle':0,
                        'prev_head':head_y,
                        'fall_angle_data':[],
                        'angle_changes':[],
                        'image':frame_filename
                    }
                people[person_id]['frames'].append((frame_id,x1,y1,x2,y2))

                # Draw bounding box & person
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.circle(frame, tuple(shoulder_center.astype(int)), 5, (255,0,0), -1)
                cv2.circle(frame, tuple(hip_center.astype(int)), 5, (0,255,0), -1)
                cv2.putText(
                        frame,
                        str(person_id),
                        (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (255,255,255), 2
                    )


                # Compute vertical drop
                torso_length = np.linalg.norm(shoulder_center - hip_center)
                if torso_length > 0:

                    # TORSO ANGLE 
                    dx = shoulder_center[0] - hip_center[0]
                    dy = shoulder_center[1] - hip_center[1]

                    angle = abs(np.degrees(np.arctan2(abs(dx), abs(dy))))
                    angle_change = abs(angle - people[person_id]['prev_angle'])
                    people[person_id]['prev_angle'] = angle
                    
                    is_horizontal = angle > 40

                    # BOX RATIO
                    box_ratio = (x2 - x1) / (y2 - y1)

                    is_wide = box_ratio > 1.2

                    # HEAD LOW
                    head_low = head_y > frame_height * 0.7

                    head_change = head_y - people[person_id]['prev_head']
                    people[person_id]['prev_head'] = head_y

                    fall_speed = head_change/dt

                    is_fall = is_horizontal and is_wide and head_low
                    print("Fall speed: ", fall_speed, is_fall)

                    angle_changed = angle_change > 10
                    people[person_id]['angle_changed'] = angle_changed

                    if angle_changed:
                        people[person_id]['fall_angle_data'].append({
                            'timestamp':timestamp_sec,
                            'frame_id':frame_id,
                            'angle':angle,
                            'angle_change':angle_change,
                            'fall_speed':fall_speed,
                            'person_id':person_id
                            })
                        fall_ongoing = True

                    people[person_id]['angle_changes'].append((angle, angle_change, is_fall))

                    if is_fall:
                        people[person_id]['fall_ongoing'] = is_fall
                        people[person_id]['fall_frame'] = frame_id
                        people[person_id]['fall_done'] = 0
                        label = "FALL [{}°]".format(angle)
                        color = (0,0,255)
                    elif angle_changed:
                        label = "TILTING [{}°]".format(angle)
                        color = (0, 165, 255)
                    else:
                        if people[person_id]['fall_ongoing']:
                            fall_done = people[person_id]['fall_done']
                            if fall_done>10:
                                fall_ongoing = False
                                # people[person_id]['fall_ongoing'] = False
                                people[person_id]['angle_changed'] = False
                                fall_angle_data = people[person_id]['fall_angle_data']
                                people[person_id]['falls'].append({
                                    'person_id':person_id,
                                    'angle_changes':fall_angle_data,
                                    'fall_frame':people[person_id]['fall_frame'],
                                    'fall_start':(fall_angle_data[0]['frame_id'],fall_angle_data[0]['timestamp']),
                                    'fall_end':(fall_angle_data[-1]['frame_id'],fall_angle_data[-1]['timestamp'])
                                })
                                people[person_id]['fall_angle_data'] = []
                                people[person_id]['fall_done'] = 0
                            else:
                                people[person_id]['fall_done'] +=1
                        label = "No Fall [{}°]".format(angle)
                        color = (0,255,0)

                    cv2.putText(
                        frame,
                        label,
                        (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, color, 2
                    )
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        if fall_ongoing:
            # Create a color overlay
            overlay = np.zeros_like(frame)
            overlay[:] = (0, 0, 255)  # Red tint

            # Blend: alpha controls transparency
            alpha = 0.3
            frame = cv2.addWeighted(frame, 1 - alpha, overlay, alpha, 0)
        out.write(frame)

    for person_id in people:
        if people[person_id]['fall_ongoing']:
            fall_angle_data = people[person_id]['fall_angle_data']
            people[person_id]['falls'].append({
                                    'person_id':person_id,
                                    'angle_changes':fall_angle_data,
                                    'fall_start':(fall_angle_data[0]['frame_id'],fall_angle_data[0]['timestamp']),
                                    'fall_end':(fall_angle_data[-1]['frame_id'],fall_angle_data[-1]['timestamp'])
                                })
        # Store each person's photo 
    cap.release()
    out.release()
    print(f"Fall detection video saved to {output_path}")
    return output_path,people

In [ ]:
output_path, people = run_fall_detection("data/Final Videos/fall-01-cam0-right-half.mp4")
output_path

In [207]:
for person_id in people:
    print(person_id)
    person = people[person_id]
    falls = person['falls']
    print("Image: ",person['image'])
    print("No of falls: ",len(falls))
    for fall in falls:
        print(fall['fall_start'],fall['fall_end'])
        for frame in fall['angle_changes']:
            # print(frame['fall_speed'])
            print(frame['frame_id'],frame['angle'],frame['fall_speed'],frame['person_id'])

3
Image:  Outputs/fall-01-cam0-right-half_people/3_2.jpg
No of falls:  1
(111.0, 3.6666666666666665) (140.0, 4.633333333333333)
111.0 75.60579 312.06204 3
116.0 56.47116 244.55931 3
123.0 3.9499733 -1061.011 3
125.0 73.1588 585.10114 3
132.0 65.095665 -33.251953 3
133.0 2.9400218 13.457793 3
134.0 58.422512 -41.577755 3
139.0 43.860653 79.4284 3
140.0 68.83916 -63.57971 3


In [102]:
for fall in falls:
    person_id = fall['person_id']
    print(f"FALL {person_id}")
    print(fall['fall_start'], fall['fall_end'],fall['person_id'])
    fall_angle_data = fall['angle_changes']
    # print(fall_angle_data)
    for angle in fall_angle_data:
        print(angle['person_id'])
        # angle['person_id']

FALL 35
3.9 4.0 35
35
35
35
35
39
35
39
35
39
35
FALL 35
4.2 4.233333333333333 35
39
35
39
35
39
35
35
FALL 35
4.566666666666666 4.566666666666666 35
35
35
35
FALL 35
4.9 4.9 35
39
35
39
35


In [105]:
for person in people:
    start_frame = people[person][0]
    end_frame = people[person][-1]
    print(person)
    print(start_frame)
    print(end_frame)

35
(2.0, 190, 48, 233, 162)
(160.0, 100, 186, 206, 240)
37
(47.0, 0, 113, 53, 226)
(48.0, 0, 113, 53, 221)
39
(98.0, 0, 113, 53, 233)
(156.0, 0, 114, 53, 232)


## Live Detection

In [ ]:
from numpy._core.numeric import True_
from sympy.logic import true
from sympy.ntheory import continued_fraction, continued_fraction_convergents


def run_fall_detection_live(input_video_path, output_filename, min_area = 2000):
    """
    Runs fall detection on a given video and writes the annotated output to the 'Outputs' folder.

    Args:
        input_video_path (str): Path to the input video file.
        output_video_name (str): (Optional) Name of the output video file. Defaults to 'fall_detection_output.mp4'.

    Returns:
        str: Path to the saved annotated video.
    """

    # Ensure Outputs directory exists
    os.makedirs("Outputs", exist_ok=True)
    # input_video_filename = os.path.basename(input_video_path)
    output_path = os.path.join("Outputs", output_filename)
    people_images_folder = os.path.splitext(output_filename)[0]
    people_output_dir = os.path.join('Outputs', people_images_folder + "/people")
    if os.path.exists(people_output_dir):
        shutil.rmtree(people_output_dir)
    os.makedirs(people_output_dir)

    fall_output_dir = os.path.join('Outputs', people_images_folder + "/falls")
    if os.path.exists(fall_output_dir):
        shutil.rmtree(fall_output_dir)
    os.makedirs(fall_output_dir)

    cap = cv2.VideoCapture(input_video_path, cv2.CAP_AVFOUNDATION)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS) or 20
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        output_path,
        fourcc,
        fps,
        (width, height)
    )

    people = {}
    prev_timestamp = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_org = frame.copy()
        frame_height, frame_width = frame.shape[:2]
        # results = model(frame)
        results = model.track(frame, persist=True, tracker='./configs/bytetrack.yaml' , conf=0.35)

        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        timestamp_sec = timestamp_ms / 1000.0
        dt = timestamp_sec - prev_timestamp
        prev_timestamp = timestamp_sec
        print(dt)

        # Get current frame position (0-indexed)
        frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
        print(f"Frame ID: {frame_id}")
        fall_ongoing = False

        for result in results:
            if result.keypoints is None:
                continue

            keypoints = result.keypoints.xy.cpu().numpy()
            boxes = result.boxes.xyxy.cpu().numpy()
            keypoints_conf = result.keypoints.conf.cpu().numpy()
            # print(keypoints_conf)
            ids = result.boxes.id
            if ids is None:
                continue
            ids = ids.cpu().numpy()
            # print(ids)

            for i, kps in enumerate(keypoints):
                box = boxes[i]
                person_id = int(ids[i])
                confs = keypoints_conf[i]
                x1, y1, x2, y2 = map(int, box)
                area = (x2 - x1) * (y2 - y1)

                if area < min_area:
                    continue

                left_shoulder = kps[5]
                right_shoulder = kps[6]

                shoulder_conf = (confs[5] + confs[6])/2
                if shoulder_conf<0.5:
                    continue

                left_hip = kps[11]
                right_hip = kps[12]
                hip_conf = (confs[11] + confs[12])/2
                if hip_conf<0.5:
                    continue

                nose = kps[0]
                left_eye = kps[1]
                right_eye = kps[2]

                head_y = np.mean([nose[1], left_eye[1], right_eye[1]])

                # Compute torso center
                shoulder_center = (left_shoulder + right_shoulder) / 2
                # print("Shoulder center confidence: ",shoulder_center[2])
                hip_center = (left_hip + right_hip) / 2

                # Identifying and storing info about the people in the frame
                if person_id not in people:
                    # INSERT_YOUR_CODE
                    frame_filename = os.path.join(people_output_dir, f"{person_id}_{int(frame_id)}.jpg")
                    person_image = frame_org.copy()
                    # Draw bounding box & person
                    cv2.rectangle(person_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.circle(person_image, tuple(shoulder_center.astype(int)), 5, (255,0,0), -1)
                    cv2.circle(person_image, tuple(hip_center.astype(int)), 5, (0,255,0), -1)
                    cv2.putText(
                            person_image,
                            str(person_id),
                            (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.9, (255,255,255), 2
                        )
                    print(frame_filename)
                    saved = cv2.imwrite(frame_filename, person_image)
                    print(saved)
                    people[person_id] = {
                        'falls': [],
                        'frames':[],
                        'fall_ongoing': False,
                        'fallen':False,
                        'fall_frame':0,
                        'fall_done':0,
                        'angle_changed':False,
                        'prev_angle':0,
                        'prev_head':head_y,
                        'fall_angle_data':[],
                        'angle_changes':[],
                        'horizontal':False,
                        'vertical':False,
                        'image':frame_filename
                    }

                # Draw bounding box & person
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.circle(frame, tuple(shoulder_center.astype(int)), 5, (255,0,0), -1)
                cv2.circle(frame, tuple(hip_center.astype(int)), 5, (0,255,0), -1)
                cv2.putText(
                        frame,
                        str(person_id),
                        (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (255,255,255), 2
                    )


                # Compute vertical drop
                torso_length = np.linalg.norm(shoulder_center - hip_center)
                if torso_length > 0:

                    # TORSO ANGLE 
                    dx = shoulder_center[0] - hip_center[0]
                    dy = shoulder_center[1] - hip_center[1]

                    angle = abs(np.degrees(np.arctan2(abs(dx), abs(dy))))
                    angle_change = abs(angle - people[person_id]['prev_angle'])
                    people[person_id]['prev_angle'] = angle
                    
                    is_horizontal = angle > 40
                    people[person_id]['horizontal']=is_horizontal

                    # BOX RATIO
                    box_ratio = (x2 - x1) / (y2 - y1)

                    is_wide = box_ratio > 1.2

                    # HEAD LOW
                    head_low = head_y > frame_height * 0.7

                    is_vertical = angle < 5 and not head_low
                    people[person_id]['vertical'] = is_vertical

                    head_change = head_y - people[person_id]['prev_head']
                    people[person_id]['prev_head'] = head_y

                    angle_changed = angle_change > 10
                    # people[person_id]['angle_changed'] = angle_changed

                    is_fall = is_horizontal and is_wide and head_low and angle_changed
                    print("Head change and angle change: ", head_change, angle_change)

                    if angle_change > 10 or head_change < -5:
                        people[person_id]['fall_ongoing'] = True
                    if people[person_id]['fall_ongoing']:
                        print("falling")
                        people[person_id]['fall_angle_data'].append({
                            'timestamp':float(timestamp_sec),
                            'frame_id':frame_id,
                            'angle':float(angle),
                            'angle_change':float(angle_change),
                            'head_y':float(head_y),
                            'shoulder_y':float(shoulder_center[1]),
                            'head_change':float(head_change),
                            })
                        
                    
                    if is_vertical:
                        if people[person_id]['fall_ongoing']:
                            print("Back up")
                        if people[person_id]['fallen']:
                            print("Recovered")
                            people[person_id]['fallen'] = False
                        people[person_id]['angle_changed'] = False
                        people[person_id]['fall_ongoing'] = False
                        people[person_id]['fall_angle_data'] = []

                    # people[person_id]['angle_changes'].append((angle, angle_change, head_change, is_fall))
                    people[person_id]['frames'].append({
                            'timestamp':float(timestamp_sec),
                            'frame_id':frame_id,
                            'horizontal':is_horizontal,
                            'vertical':is_vertical,
                            'angle':float(angle),
                            'angle_change':float(angle_change),
                            'head_y':float(head_y),
                            'shoulder_y':float(shoulder_center[1]),
                            'head_change':float(head_change),
                            'fall_ongoing':people[person_id]['fall_ongoing'],
                            })

                    if people[person_id]['horizontal'] and people[person_id]['fallen']:
                        continue

                    if is_fall:
                        print("Fell")
                        people[person_id]['fallen'] = True
                        people[person_id]['fall_frame'] = frame_id
                        fall_ongoing = True
                        fall_angle_data = people[person_id]['fall_angle_data']
                        dy = fall_angle_data[-1]['timestamp'] - fall_angle_data[0]['timestamp']
                        if dy == 0:
                            dy = 0.0001
                        
                        accel_head = []
                        # INSERT_YOUR_CODE
                        # Calculate acceleration of head from fall_angle_data
                        # We'll use the difference of head_change between consecutive frames divided by the time difference
                        if len(fall_angle_data) > 2:
                            for i in range(1, len(fall_angle_data)):
                                dt = fall_angle_data[i]['timestamp'] - fall_angle_data[i-1]['timestamp']
                                if dt == 0:
                                    accel_head.append(0)
                                else:
                                    delta_v = fall_angle_data[i]['head_change'] - fall_angle_data[i-1]['head_change']
                                    accel = delta_v / dt
                                    accel_head.append(accel)
                        else:
                            # Not enough data for meaningful acceleration
                            accel_head = []
                        accel_head = sum(accel_head)/len(accel_head)
                            
                        fall_data = {
                            'fall_id':len(people[person_id]['falls']),
                            'no_of_people':len(ids),
                            'person_id':str(person_id),
                            'angle_changes':fall_angle_data,
                            'fall_frame':people[person_id]['fall_frame'],
                            'head_speed':(fall_angle_data[0]['head_y'] - fall_angle_data[-1]['head_y'])/dy,
                            'head_acceleration':accel_head,
                            'shoulder_speed':(fall_angle_data[0]['shoulder_y'] - fall_angle_data[-1]['shoulder_y'])/dy,
                            'fall_start':(fall_angle_data[0]['frame_id'],fall_angle_data[0]['timestamp']),
                            'fall_end':(fall_angle_data[-1]['frame_id'],fall_angle_data[-1]['timestamp'])
                        }
                        people[person_id]['falls'].append(fall_data)
                        fall_filename = str(person_id) + "_" + str(len(people[person_id]['falls'])) + '.json'
                        with open(os.path.join(fall_output_dir, fall_filename), 'w') as file:
                            json.dump(json.dumps(fall_data), file, indent=4)
                        people[person_id]['fall_angle_data'] = []
                        people[person_id]['fall_ongoing'] = False
                        label = "FALL [{}°]".format(angle)
                        color = (0,0,255)
                    elif angle_changed:
                        label = "FALLING [{}°]".format(angle)
                        color = (0, 165, 255)
                    else:
                        label = "No Fall [{}°]".format(angle)
                        color = (0,255,0)

                    cv2.putText(
                        frame,
                        label,
                        (x1-5, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, color, 2
                    )
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        if fall_ongoing:
            # Create a color overlay
            overlay = np.zeros_like(frame)
            overlay[:] = (0, 0, 255)  # Red tint

            # Blend: alpha controls transparency
            alpha = 0.3
            frame = cv2.addWeighted(frame, 1 - alpha, overlay, alpha, 0)
        out.write(frame)
        print('here')
        cv2.imshow("Live Fall Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
           break

    # for person_id in people:
    #     if people[person_id]['fall_ongoing']:
    #         fall_angle_data = people[person_id]['fall_angle_data']
    #         people[person_id]['falls'].append({
    #                                 'person_id':person_id,
    #                                 'angle_changes':fall_angle_data,
    #                                 'fall_start':(fall_angle_data[0]['frame_id'],fall_angle_data[0]['timestamp']),
    #                                 'fall_end':(fall_angle_data[-1]['frame_id'],fall_angle_data[-1]['timestamp'])
    #                             })
        # Store each person's photo 
    cap.release()
    out.release()
    print(f"Fall detection video saved to {output_path}")
    return output_path,people

In [ ]:
output_dir, people = run_fall_detection_live(0, output_filename='test0.mp4')


0: 384x640 1 person, 186.2ms
Speed: 13.1ms preprocess, 186.2ms inference, 33.5ms postprocess per image at shape (1, 3, 384, 640)
0.0
Frame ID: 0.0
here

0: 384x640 1 person, 59.9ms
Speed: 2.5ms preprocess, 59.9ms inference, 17.1ms postprocess per image at shape (1, 3, 384, 640)
0.0
Frame ID: 0.0
here

0: 384x640 1 person, 35.8ms
Speed: 1.9ms preprocess, 35.8ms inference, 18.5ms postprocess per image at shape (1, 3, 384, 640)
0.0
Frame ID: 0.0
here

0: 384x640 1 person, 35.5ms
Speed: 1.7ms preprocess, 35.5ms inference, 17.4ms postprocess per image at shape (1, 3, 384, 640)
0.0
Frame ID: 0.0
here

0: 384x640 1 person, 36.4ms
Speed: 1.8ms preprocess, 36.4ms inference, 17.5ms postprocess per image at shape (1, 3, 384, 640)
0.0
Frame ID: 0.0
here

0: 384x640 1 person, 35.4ms
Speed: 2.0ms preprocess, 35.4ms inference, 18.2ms postprocess per image at shape (1, 3, 384, 640)
0.0
Frame ID: 0.0
here

0: 384x640 1 person, 42.0ms
Speed: 2.2ms preprocess, 42.0ms inference, 39.1ms postprocess per im

KeyboardInterrupt: 

: 

In [143]:
people[51]['falls']

[{'fall_id': 0,
  'person_id': '51',
  'angle_changes': [{'timestamp': 2.8666666666666667,
    'frame_id': 87.0,
    'angle': 81.18475341796875,
    'angle_change': 10.3150634765625,
    'head_y': 218.94483947753906,
    'shoulder_y': 228.23509216308594,
    'head_change': -7.8128509521484375}],
  'fall_frame': 87.0,
  'head_speed': 0.0,
  'shoulder_speed': 0.0,
  'fall_start': (87.0, 2.8666666666666667),
  'fall_end': (87.0, 2.8666666666666667)}]

In [148]:
for frame in people[51]['frames'][80:]:
    print(frame)

{'timestamp': 2.8, 'frame_id': 85.0, 'horizontal': np.True_, 'vertical': np.False_, 'angle': 78.80101776123047, 'angle_change': 2.0325469970703125, 'head_y': 224.60008239746094, 'shoulder_y': 236.91551208496094, 'head_change': 4.770416259765625, 'fall_ongoing': False}
{'timestamp': 2.8333333333333335, 'frame_id': 86.0, 'horizontal': np.True_, 'vertical': np.False_, 'angle': 70.86968994140625, 'angle_change': 7.931327819824219, 'head_y': 226.7576904296875, 'shoulder_y': 239.67813110351562, 'head_change': 2.1576080322265625, 'fall_ongoing': False}
{'timestamp': 2.8666666666666667, 'frame_id': 87.0, 'horizontal': np.True_, 'vertical': np.False_, 'angle': 81.18475341796875, 'angle_change': 10.3150634765625, 'head_y': 218.94483947753906, 'shoulder_y': 228.23509216308594, 'head_change': -7.8128509521484375, 'fall_ongoing': True}
{'timestamp': 2.9, 'frame_id': 88.0, 'horizontal': np.True_, 'vertical': np.False_, 'angle': 69.84111785888672, 'angle_change': 11.343635559082031, 'head_y': 218.119